In [ ]:
from dataclasses import dataclass
from pathlib import Path
from typing import (
    Any,
    Dict,
    Iterator,
    List,
    NamedTuple,
    Optional,
    Self,
)

from pydantic import BaseModel, Field

from tnh_scholar.logging_config import get_child_logger
from tnh_scholar.metadata.metadata import Frontmatter, Metadata, ProcessMetadata
from tnh_scholar.text_processing import NumberedText
from tnh_scholar.utils.file_utils import get_text_from_file
from tnh_scholar.utils.lang import get_language_code

logger = get_child_logger(__name__)

# Core models
class SectionRange(NamedTuple):
    """Represents the line range of a section."""
    start: int  # Start line (inclusive)
    end: int    # End line (Exclusive)
    
class SectionEntry(NamedTuple):
    """Represents a section with its content during iteration."""
    number: int         # Logical Section number (1 based index)
    title: str          # Section title 
    content: str        # Section content
    range: SectionRange # Section range

class LogicalSection(BaseModel):
    """
    Represents a contextually meaningful segment of a larger text.
    
    Sections should preserve natural breaks in content 
    (explicit section markers, topic shifts, argument development, narrative progression) 
    while staying within specified size limits in order to create chunks suitable for AI processing.
    """  # noqa: E501
    start_line: int = Field(
        ..., 
        description="Starting line number that begins this logical segment"
    )
    title: str = Field(
        ...,
        description="Descriptive title of section's key content"
    )

            
class AIResponse(BaseModel):
    """Text Object for dividing large texts into AI-processable segments while
    maintaining broader document context."""
    document_summary: str = Field(
        ...,
        description="Concise, comprehensive overview of the text's content and purpose"
    )
    document_metadata: str = Field(
        ...,
        description="Available Dublin Core standard metadata in human-readable format"
    )
    key_concepts: str = Field(
        ...,
        description="Important terms, ideas, or references that appear throughout the text"  # noqa: E501
    )
    narrative_context: str = Field(
        ...,
        description="Concise overview of how the text develops or progresses as a whole"
    )
    language: str = Field(..., description="ISO 639-1 language code")
    sections: List[LogicalSection]
    
@dataclass
class SectionObject:
    """Represents a section of text with metadata."""
    title: str
    section_range: SectionRange
    metadata: Optional[Metadata] 

    @classmethod
    def from_logical_section(
        cls, 
        logical_section: LogicalSection, 
        end_line: int, 
        metadata: Optional[Metadata] = None
        ) -> "SectionObject":
        """Create a SectionObject from a LogicalSection model."""
        return cls(
            title=logical_section.title,
            section_range = SectionRange(logical_section.start_line, end_line),
            metadata = metadata 
        )

# This represents the serializable state of a TextObject
class TextObjectInfo(BaseModel):
    """Serializable information about a text and its sections."""
    source_file: Optional[Path] = None  # Original text file path
    language: str
    sections: List[SectionObject]
    # mdata: Dict[str, Any]

In [ ]:
from dataclasses import dataclass
from pathlib import Path
from typing import (
    Any,
    Dict,
    Iterator,
    List,
    NamedTuple,
    Optional,
    Self,
)

from pydantic import BaseModel, Field

class TextObjectInfo(BaseModel):
    """Serializable information about a text and its sections."""
    source_file: Optional[Path] = None  # Original text file path
    language: str
    sections: List[SectionObject]
    mdata: Dict[str, Any]